# Let's test both databases

We will pitch the databases against one another to see how they differ.

## Let's setup first

In [ ]:
from IPython.display import clear_output

for i in range(10):
    clear_output()
    
import psycopg2
conn = psycopg2.connect("dbname=dota user=dota password=dota host=psql")
postgres = conn.cursor()

## Simple query
Postgres:

In [ ]:
postgres.execute("SELECT * FROM heroes WHERE name = 'Tiny'")
postgres.fetchone()

MongoDB:

In [ ]:
from pymongo import MongoClient
from pprint import pprint
client = MongoClient('mongodb', 27017)
mongodb = client.dota

In [ ]:
pprint(mongodb.heroes.find_one({"name": "Tiny"}))